In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train0 = pd.read_csv(r'C:\Users\pritam\Downloads\train_v9rqX0R.csv')
test0 = pd.read_csv(r'C:\Users\pritam\Downloads\test_AbJTz2l.csv')
train0.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
test0.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [4]:
print(train0.shape)
print(test0.shape)

(8523, 12)
(5681, 11)


In [5]:
train0.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [6]:
test0.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [7]:
data = pd.concat([train0,test0], axis=0).reset_index().drop('index', axis=1)


In [8]:
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1,NaN
14200,FDD47,7.60,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2,NaN
14201,NCO17,10.00,Low Fat,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,Tier 2,Supermarket Type1,NaN
14202,FDJ26,15.30,Regular,0.000000,Canned,214.6218,OUT017,2007,NaN,Tier 2,Supermarket Type1,NaN


In [9]:
data['Item_Weight'].fillna(data['Item_Weight'].mean(), inplace=True)
data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0], inplace=True)

In [10]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [11]:
data['Item_Fat_Content'].value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [12]:
fat_dict = {'LF':'Low Fat', 'reg':'Regular','low fat':'Low Fat'}
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace(fat_dict)

In [13]:
data['Outlet_Age'] = 2022 - data['Outlet_Establishment_Year']

In [14]:
data.drop('Outlet_Establishment_Year', axis=1, inplace= True)

In [15]:
data1 = data.copy()

In [16]:
data1.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Age
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,Medium,Tier 1,Supermarket Type1,3735.1380,23
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.4228,13
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.2700,23
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,Medium,Tier 3,Grocery Store,732.3800,24
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.7052,35


In [78]:
train1 = data1.iloc[:train0.shape[0],]
test1 = data1.iloc[train0.shape[0]:,]

In [79]:
print(train1.shape)
print(test1.shape)

(8523, 12)
(5681, 12)


In [80]:
target_col = train1['Item_Outlet_Sales']
train2 = train1.copy()
test2 = test1.copy()

In [81]:
train2.drop('Item_Outlet_Sales', axis=1, inplace=True)
num_cols = train2.select_dtypes(include='number').columns

In [82]:
train2.drop('Item_Identifier', axis=1, inplace=True)
train2 = pd.get_dummies(train2)
train2

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Age,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.300,0.016047,249.8092,23,1,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
1,5.920,0.019278,48.2692,13,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
2,17.500,0.016760,141.6180,23,1,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,19.200,0.000000,182.0950,24,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
4,8.930,0.000000,53.8614,35,1,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0.056783,214.5218,35,1,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
8519,8.380,0.046982,108.1570,20,0,1,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
8520,10.600,0.035186,85.1224,18,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
8521,7.210,0.145221,103.1332,13,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0


In [83]:
scaler = MinMaxScaler()
scaler.fit(train2)
scaled_train2 = scaler.transform(train2)

# scaled_target = scaler.fit_transform(target_col.values.reshape(-1,1))

In [84]:
X_train,X_test,y_train, y_test = train_test_split(scaled_train2, target_col, test_size=0.2, random_state=123)

xgb_model = xgb.XGBRegressor(objective='reg:linear', n_estimators=10, seed=123)
xgb_model.fit(X_train,y_train)
pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test,pred)
rmse = np.sqrt(mse)
rmse


[22:18:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


1078.9696570293866

In [76]:
test2 = test2.reset_index(drop=True)

In [86]:
test2.drop(['Item_Identifier','Item_Outlet_Sales'], axis=1, inplace=True)
test2 = pd.get_dummies(test2)
test2

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Age,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
8523,20.750000,0.007565,107.8622,23,1,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
8524,8.300000,0.038428,87.3198,15,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
8525,14.600000,0.099575,241.7538,24,1,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
8526,7.315000,0.015388,155.0340,15,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
8527,12.792854,0.118599,234.2300,37,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,10.500000,0.013496,141.3154,25,0,1,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
14200,7.600000,0.142991,169.1448,13,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
14201,10.000000,0.073529,118.7440,20,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
14202,15.300000,0.000000,214.6218,15,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0


In [89]:
scaled_test2 = scaler.transform(test2)
test_pred = xgb_model.predict(scaled_test2)

In [90]:
sub_df = pd.DataFrame()
sub_df['Item_Identifier'] = test0['Item_Identifier']
sub_df['Outlet_Identifier'] = test0['Outlet_Identifier']
sub_df['Item_Outlet_Sales'] = test_pred

sub_df.to_csv('xgboost_sub1.csv', index=False)